###### Imports and Settings

In [1]:
import pandas as pd
import numpy as np
import requests
from functools import reduce
import matplotlib.pyplot as plt
import pickle
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
import sys
sys.path.append("../../../Functions and Dictionaries") # Adds higher directory to python modules path
import geodict
namestocommon = geodict.namestocommon
geotogeoid = geodict.geotogeoid
tofullcensus = geodict.tofullcensus
import sqlite3 as sq

# This notebook outlines the download and formatting process for the University of Wisconsin's Population Health Institute's County Health Rankings for counties places in the GNRC operating region.  

Go to this page: https://www.countyhealthrankings.org/explore-health-rankings/tennessee/data-and-resources  
Download the "(YEAR) Tennessee Data" excel workbook  
If the new version is the same as the 2022 download, then it will contain the following pages:  
+ Introduction, Outcomes & Factors Rankings, Outcomes & Factors SubRankings, Ranked Measure Data, Additional Measure Data, Ranked Measure Sources & Years, Additional Measure Sources & Years  
+ For both "Outcomes & Factors Rankings", "Outcomes & Factors SubRankings", and Additional Measure Data": unmerge headers and keep the FIPS, rename the County to "NAME", delete the State column, and keep only the ranks (we won't use the z-scores), and make sure the column headers are consistent with the outcomes or factors. Ensure the counties are alphabetized and move all of the rankings into one sheet.
  

Save these csvs as they come in the Data Downloads folder of Parent Data Gathering  


# I've got 2019 - 2022 formatted.


## Merge Multiple Years

In [2]:
#import and drop FIPS on the import as we'll join it with the custom module
data = pd.read_csv('../../Data Downloads/CountyHealthRankings_2022.csv', dtype = str)
data.head(3)

,NAME,Year,Health Outcomes Ranking,Health Factors Ranking,Length of Life Ranking,Quality of Life Ranking,Health Behaviors Ranking,Clinical Care Ranking,Social & Economic Factors Ranking,Physical Environment Ranking,Life Expectancy,95% CI - Low,95% CI - High,Life Expectancy (AIAN),Life Expectancy (AIAN) 95% CI - Low,Life Expectancy (AIAN) 95% CI - High,Life Expectancy (Asian),Life Expectancy (Asian) 95% CI - Low,Life Expectancy (Asian) 95% CI - High,Life Expectancy (Black),Life Expectancy (Black) 95% CI - Low,Life Expectancy (Black) 95% CI - High,Life Expectancy (Hispanic),Life Expectancy (Hispanic) 95% CI - Low,Life Expectancy (Hispanic) 95% CI - High,Life Expectancy (White),Life Expectancy (White) 95% CI - Low,Life Expectancy (White) 95% CI - High,# Deaths,Age-Adjusted Death Rate,95% CI - Low.1,95% CI - High.1,Age-Adjusted Mortality (AIAN),Age-Adjusted Mortality (AIAN) 95% CI - Low,Age-Adjusted Mortality (AIAN) 95% CI - High,Age-Adjusted Mortality (Asian),Age-Adjusted Mortality (Asian) 95% CI - Low,Age-Adjusted Mortality (Asian) 95% CI - High,Age-Adjusted Mortality (Black),Age-Adjusted Mortality (Black) 95% CI - Low,Age-Adjusted Mortality (Black) 95% CI - High,Age-Adjusted Mortality (Hispanic),Age-Adjusted Mortality (Hispanic) 95% CI - Low,Age-Adjusted Mortality (Hispanic) 95% CI - High,Age-Adjusted Mortality (White),Age-Adjusted Mortality (White) 95% CI - Low,Age-Adjusted Mortality (White) 95% CI - High,# Child Deaths,Child Mortality Rate,95% CI - Low.2,95% CI - High.2,Child Mortality Rate (AIAN),Child Mortality Rate (AIAN) 95% CI - Low,Child Mortality Rate (AIAN) 95% CI - High,Child Mortality Rate (Asian),Child Mortality Rate (Asian) 95% CI - Low,Child Mortality Rate (Asian) 95% CI - High,Child Mortality Rate (Black),Child Mortality Rate (Black) 95% CI - Low,Child Mortality Rate (Black) 95% CI - High,Child Mortality Rate (Hispanic),Child Mortality Rate (Hispanic) 95% CI - Low,Child Mortality Rate (Hispanic) 95% CI - High,Child Mortality Rate (White),Child Mortality Rate (White) 95% CI - Low,Child Mortality Rate (White) 95% CI - High,# Infant Deaths,Infant Mortality Rate,95% CI - Low.3,95% CI - High.3,Infant Mortality Rate (AIAN),Infant Mortality Rate (AIAN) 95% CI - Low,Infant Mortality Rate (AIAN) 95% CI - High,Infant Mortality Rate (Asian),Infant Mortality Rate (Asian) 95% CI - Low,Infant Mortality Rate (Asian) 95% CI - High,Infant Mortality Rate (Black),Infant Mortality Rate (Black) 95% CI - Low,Infant Mortality Rate (Black) 95% CI - High,Infant Mortality Rate (Hispanic),Infant Mortality Rate (Hispanic) 95% CI - Low,Infant Mortality Rate (Hispanic) 95% CI - High,Infant Mortality Rate (White),Infant Mortality Rate (White) 95% CI - Low,Infant Mortality Rate (White) 95% CI - High,% Frequent Physical Distress,95% CI - Low.4,95% CI - High.4,% Frequent Mental Distress,95% CI - Low.5,95% CI - High.5,% Adults with Diabetes,95% CI - Low.6,95% CI - High.6,# HIV Cases,HIV Prevalence Rate,# Food Insecure,% Food Insecure,# Limited Access to Healthy Foods,% Limited Access to Healthy Foods,# Drug Overdose Deaths,Drug Overdose Mortality Rate,95% CI - Low.7,95% CI - High.7,Drug Overdose Mortality Rate (AIAN),Drug Overdose Mortality Rate (AIAN) 95% CI - Low,Drug Overdose Mortality Rate (AIAN) 95% CI - High,Drug Overdose Mortality Rate (Asian),Drug Overdose Mortality Rate (Asian) 95% CI - Low,Drug Overdose Mortality Rate (Asian) 95% CI - High,Drug Overdose Mortality Rate (Black),Drug Overdose Mortality Rate (Black) 95% CI - Low,Drug Overdose Mortality Rate (Black) 95% CI - High,Drug Overdose Mortality Rate (Hispanic),Drug Overdose Mortality Rate (Hispanic) 95% CI - Low,Drug Overdose Mortality Rate (Hispanic) 95% CI - High,Drug Overdose Mortality Rate (White),Drug Overdose Mortality Rate (White) 95% CI - Low,Drug Overdose Mortality Rate (White) 95% CI - High,# Motor Vehicle Deaths,Motor Vehicle Mortality Rate,95% CI - Low.8,95% CI - High.8,MV Mortality Rate (AIAN),MV Mortality Rate (AIAN) 95% CI - Low,MV Mortality Rate (AIAN) 95% CI - High,

In [3]:
data.set_index(['NAME', 'Year'], inplace = True)

In [4]:
df = pd.read_csv('../../Data Downloads/CountyHealthRankings_2023.csv', dtype = str)
df.head(3)

,NAME,Year,Health Outcomes Ranking,Health Factors Ranking,Length of Life Ranking,Quality of Life Ranking,Health Behaviors Ranking,Clinical Care Ranking,Social & Economic Factors Ranking,Physical Environment Ranking,Life Expectancy,Life Expectancy (AIAN),Life Expectancy (Black),Life Expectancy (Hispanic),Life Expectancy (White),# Deaths,Age-Adjusted Death Rate,Age-Adjusted Mortality (AIAN),Age-Adjusted Mortality (Asian),Age-Adjusted Mortality (Black),Age-Adjusted Mortality (Hispanic),Age-Adjusted Mortality (White),# Child Deaths,Child Mortality Rate,Child Mortality Rate (AIAN),Child Mortality Rate (Asian),Child Mortality Rate (Black),Child Mortality Rate (Hispanic),Child Mortality Rate (White),# Infant Deaths,Infant Mortality Rate,Infant Mortality Rate (AIAN),Infant Mortality Rate (Asian),Infant Mortality Rate (Black),Infant Mortality Rate (Hispanic),Infant Mortality Rate (White),% Frequent Physical Distress,% Frequent Mental Distress,% Adults with Diabetes,# HIV Cases,HIV Prevalence Rate,# Food Insecure,% Food Insecure,# Limited Access to Healthy Foods,% Limited Access to Healthy Foods,# Drug Overdose Deaths,Drug Overdose Mortality Rate,Drug Overdose Mortality Rate (AIAN),Drug Overdose Mortality Rate (Asian),Drug Overdose Mortality Rate (Black),Drug Overdose Mortality Rate (Hispanic),Drug Overdose Mortality Rate (White),% Insufficient Sleep,# Uninsured Adults,% Uninsured Adults,# Uninsured Children,% Uninsured Children,Other Primary Care Provider Rate,Other Primary Care Provider Ratio,Cohort Size,High School Graduation Rate,% Disconnected Youth,Average Grade Performance for 3rd Graders on English Language Arts Standardized Tests,Average Grade Performance for 3rd Graders on English Language Arts Standardized Tests (Asian),Average Grade Performance for 3rd Graders on English Language Arts Standardized Tests (Black),Average Grade Performance for 3rd Graders on English Language Arts Standardized Tests (Hispanic),Average Grade Performance for 3rd Graders on English Language Arts Standardized Tests (White),Average Grade Performance for 3rd Graders on Math Standardized Tests,Average Grade Performance for 3rd Graders on Math Standardized Tests (Asian),Average Grade Performance for 3rd Graders on Math Standardized Tests (Black),Average Grade Performance for 3rd Graders on Math Standardized Tests (Hispanic),Average Grade Performance for 3rd Graders on Math Standardized Tests (White),Segregation Index,Spending per Pupil,School Funding Adequacy,Women's Median Earnings,Men's Median Earnings,Gender Pay Gap,% Enrolled in Free or Reduced Lunch,% Household Income Required for Child Care Expenses,# Child Care Centers,"Child Care Centers per 1,000 Children",# Suicide Deaths,Suicide Rate (Age-Adjusted),Crude Suicide Rate,Suicide Rate (AIAN),Suicide Rate (Asian),Suicide Rate (Black),Suicide Rate (Hispanic),Suicide Rate (White),# Firearm Fatalities,Firearm Fatalities Rate,Firearm Fatalities Rate (AIAN),Firearm Fatalities Rate (Asian),Firearm Fatalities Rate (Black),Firearm Fatalities Rate (Hispanic),Firearm Fatalities Rate (White),# Motor Vehicle Deaths,Motor Vehicle Mortality Rate,MV Mortality Rate (AIAN),MV Mortality Rate (Asian),MV Mortality Rate (Black),MV Mortality Rate (Hispanic),MV Mortality Rate (White),Non-Petitioned Cases,Petitioned Cases,Denominator,Juvenile Arrest Rate,% Voter Turnout,% Census Participation,Traffic Volume,Premature Deaths,Years of Potential Life Lost Rate,YPLL Rate (AIAN),YPLL Rate (Asian),YPLL Rate (Black),YPLL Rate (Hispanic),YPLL Rate (White),% Fair or Poor Health,Average Number of Physically Unhealthy Days,Average Number of Mentally Unhealthy Days,% Low Birthweight,% LBW (AIAN),% LBW (Asian),% LBW (Black),% LBW (Hispanic),% LBW (White),% Adults Reporting Currently Smoking,% Adults with Obesity,Food Environment Index,% Physically Inactive,% With Access to Exercise Opportunities,% Excessive Drinking,# Alcohol-Impaired Driving Deaths,# Driving Deaths,% Driving Deaths with Alcohol Involvement,# Chlamydia Cases,Chlamydia

In [5]:
df.set_index(['NAME', 'Year'], inplace = True)
colstt = df.columns

In [6]:
data = data.filter(colstt, axis = 1)

In [7]:
dfs = [data, df]
data = pd.concat(dfs)
data.head()

,,Health Outcomes Ranking,Health Factors Ranking,Length of Life Ranking,Quality of Life Ranking,Health Behaviors Ranking,Clinical Care Ranking,Social & Economic Factors Ranking,Physical Environment Ranking,Life Expectancy,Life Expectancy (AIAN),Life Expectancy (Black),Life Expectancy (Hispanic),Life Expectancy (White),# Deaths,Age-Adjusted Death Rate,Age-Adjusted Mortality (AIAN),Age-Adjusted Mortality (Asian),Age-Adjusted Mortality (Black),Age-Adjusted Mortality (Hispanic),Age-Adjusted Mortality (White),# Child Deaths,Child Mortality Rate,Child Mortality Rate (AIAN),Child Mortality Rate (Asian),Child Mortality Rate (Black),Child Mortality Rate (Hispanic),Child Mortality Rate (White),# Infant Deaths,Infant Mortality Rate,Infant Mortality Rate (AIAN),Infant Mortality Rate (Asian),Infant Mortality Rate (Black),Infant Mortality Rate (Hispanic),Infant Mortality Rate (White),% Frequent Physical Distress,% Frequent Mental Distress,% Adults with Diabetes,# HIV Cases,HIV Prevalence Rate,# Food Insecure,% Food Insecure,# Limited Access to Healthy Foods,% Limited Access to Healthy Foods,# Drug Overdose Deaths,Drug Overdose Mortality Rate,Drug Overdose Mortality Rate (AIAN),Drug Overdose Mortality Rate (Asian),Drug Overdose Mortality Rate (Black),Drug Overdose Mortality Rate (Hispanic),Drug Overdose Mortality Rate (White),% Insufficient Sleep,# Uninsured Adults,% Uninsured Adults,# Uninsured Children,% Uninsured Children,Other Primary Care Provider Rate,Other Primary Care Provider Ratio,Cohort Size,High School Graduation Rate,% Disconnected Youth,Average Grade Performance for 3rd Graders on English Language Arts Standardized Tests,Average Grade Performance for 3rd Graders on English Language Arts Standardized Tests (Asian),Average Grade Performance for 3rd Graders on English Language Arts Standardized Tests (Black),Average Grade Performance for 3rd Graders on English Language Arts Standardized Tests (Hispanic),Average Grade Performance for 3rd Graders on English Language Arts Standardized Tests (White),Average Grade Performance for 3rd Graders on Math Standardized Tests,Average Grade Performance for 3rd Graders on Math Standardized Tests (Asian),Average Grade Performance for 3rd Graders on Math Standardized Tests (Black),Average Grade Performance for 3rd Graders on Math Standardized Tests (Hispanic),Average Grade Performance for 3rd Graders on Math Standardized Tests (White),Segregation Index,Spending per Pupil,School Funding Adequacy,Women's Median Earnings,Men's Median Earnings,Gender Pay Gap,% Enrolled in Free or Reduced Lunch,% Household Income Required for Child Care Expenses,# Child Care Centers,# Suicide Deaths,Suicide Rate (Age-Adjusted),Crude Suicide Rate,Suicide Rate (AIAN),Suicide Rate (Asian),Suicide Rate (Black),Suicide Rate (Hispanic),Suicide Rate (White),# Firearm Fatalities,Firearm Fatalities Rate,Firearm Fatalities Rate (AIAN),Firearm Fatalities Rate (Asian),Firearm Fatalities Rate (Black),Firearm Fatalities Rate (Hispanic),Firearm Fatalities Rate (White),# Motor Vehicle Deaths,Motor Vehicle Mortality Rate,MV Mortality Rate (AIAN),MV Mortality Rate (Asian),MV Mortality Rate (Black),MV Mortality Rate (Hispanic),MV Mortality Rate (White),Non-Petitioned Cases,Petitioned Cases,Denominator,Juvenile Arrest Rate,Traffic Volume,Premature Deaths,Years of Potential Life Lost Rate,YPLL Rate (AIAN),YPLL Rate (Asian),YPLL Rate (Black),YPLL Rate (Hispanic),YPLL Rate (White),% Fair or Poor Health,Average Number of Physically Unhealthy Days,Average Number of Mentally Unhealthy Days,% Low Birthweight,% LBW (AIAN),% LBW (Asian),% LBW (Black),% LBW (Hispanic),% LBW (White),% Adults Reporting Currently Smoking,% Adults with Obesity,Food Environment Index,% Physically Inactive,% With Access to Exercise Opportunities,% Excessive Drinking,# Alcohol-Impaired Driving Deaths,# Driving Deaths,% Driving Deaths with Alcohol Involvement,# Chlamydia Cases,Chlamydia Rate,Teen Birth Rate,Teen Birth Rate (AIAN),Teen Birth Rate (Asian),Teen Birth Rate (Bl

In [8]:
twentyone = pd.read_csv('../../Data Downloads/CountyHealthRankings_2021.csv', dtype = str)
twentyone.set_index(['NAME', 'Year'], inplace = True)

In [9]:
twentyone = twentyone.filter(colstt, axis = 1)

In [10]:
dfs = [data, twentyone]
data = pd.concat(dfs)
data.head(2)

,,Health Outcomes Ranking,Health Factors Ranking,Length of Life Ranking,Quality of Life Ranking,Health Behaviors Ranking,Clinical Care Ranking,Social & Economic Factors Ranking,Physical Environment Ranking,Life Expectancy,Life Expectancy (AIAN),Life Expectancy (Black),Life Expectancy (Hispanic),Life Expectancy (White),# Deaths,Age-Adjusted Death Rate,Age-Adjusted Mortality (AIAN),Age-Adjusted Mortality (Asian),Age-Adjusted Mortality (Black),Age-Adjusted Mortality (Hispanic),Age-Adjusted Mortality (White),# Child Deaths,Child Mortality Rate,Child Mortality Rate (AIAN),Child Mortality Rate (Asian),Child Mortality Rate (Black),Child Mortality Rate (Hispanic),Child Mortality Rate (White),# Infant Deaths,Infant Mortality Rate,Infant Mortality Rate (AIAN),Infant Mortality Rate (Asian),Infant Mortality Rate (Black),Infant Mortality Rate (Hispanic),Infant Mortality Rate (White),% Frequent Physical Distress,% Frequent Mental Distress,% Adults with Diabetes,# HIV Cases,HIV Prevalence Rate,# Food Insecure,% Food Insecure,# Limited Access to Healthy Foods,% Limited Access to Healthy Foods,# Drug Overdose Deaths,Drug Overdose Mortality Rate,Drug Overdose Mortality Rate (AIAN),Drug Overdose Mortality Rate (Asian),Drug Overdose Mortality Rate (Black),Drug Overdose Mortality Rate (Hispanic),Drug Overdose Mortality Rate (White),% Insufficient Sleep,# Uninsured Adults,% Uninsured Adults,# Uninsured Children,% Uninsured Children,Other Primary Care Provider Rate,Other Primary Care Provider Ratio,Cohort Size,High School Graduation Rate,% Disconnected Youth,Average Grade Performance for 3rd Graders on English Language Arts Standardized Tests,Average Grade Performance for 3rd Graders on English Language Arts Standardized Tests (Asian),Average Grade Performance for 3rd Graders on English Language Arts Standardized Tests (Black),Average Grade Performance for 3rd Graders on English Language Arts Standardized Tests (Hispanic),Average Grade Performance for 3rd Graders on English Language Arts Standardized Tests (White),Average Grade Performance for 3rd Graders on Math Standardized Tests,Average Grade Performance for 3rd Graders on Math Standardized Tests (Asian),Average Grade Performance for 3rd Graders on Math Standardized Tests (Black),Average Grade Performance for 3rd Graders on Math Standardized Tests (Hispanic),Average Grade Performance for 3rd Graders on Math Standardized Tests (White),Segregation Index,Spending per Pupil,School Funding Adequacy,Women's Median Earnings,Men's Median Earnings,Gender Pay Gap,% Enrolled in Free or Reduced Lunch,% Household Income Required for Child Care Expenses,# Child Care Centers,# Suicide Deaths,Suicide Rate (Age-Adjusted),Crude Suicide Rate,Suicide Rate (AIAN),Suicide Rate (Asian),Suicide Rate (Black),Suicide Rate (Hispanic),Suicide Rate (White),# Firearm Fatalities,Firearm Fatalities Rate,Firearm Fatalities Rate (AIAN),Firearm Fatalities Rate (Asian),Firearm Fatalities Rate (Black),Firearm Fatalities Rate (Hispanic),Firearm Fatalities Rate (White),# Motor Vehicle Deaths,Motor Vehicle Mortality Rate,MV Mortality Rate (AIAN),MV Mortality Rate (Asian),MV Mortality Rate (Black),MV Mortality Rate (Hispanic),MV Mortality Rate (White),Non-Petitioned Cases,Petitioned Cases,Denominator,Juvenile Arrest Rate,Traffic Volume,Premature Deaths,Years of Potential Life Lost Rate,YPLL Rate (AIAN),YPLL Rate (Asian),YPLL Rate (Black),YPLL Rate (Hispanic),YPLL Rate (White),% Fair or Poor Health,Average Number of Physically Unhealthy Days,Average Number of Mentally Unhealthy Days,% Low Birthweight,% LBW (AIAN),% LBW (Asian),% LBW (Black),% LBW (Hispanic),% LBW (White),% Adults Reporting Currently Smoking,% Adults with Obesity,Food Environment Index,% Physically Inactive,% With Access to Exercise Opportunities,% Excessive Drinking,# Alcohol-Impaired Driving Deaths,# Driving Deaths,% Driving Deaths with Alcohol Involvement,# Chlamydia Cases,Chlamydia Rate,Teen Birth Rate,Teen Birth Rate (AIAN),Teen Birth Rate (Asian),Teen Birth Rate (Bl

In [11]:
twenty = pd.read_csv('../../Data Downloads/CountyHealthRankings_2020.csv', dtype = str)
twenty.set_index(['NAME', 'Year'], inplace = True)

In [12]:
twenty = twenty.filter(colstt, axis = 1)

In [13]:
dfs = [data, twenty]
data = pd.concat(dfs)
data.tail(2)

,,Health Outcomes Ranking,Health Factors Ranking,Length of Life Ranking,Quality of Life Ranking,Health Behaviors Ranking,Clinical Care Ranking,Social & Economic Factors Ranking,Physical Environment Ranking,Life Expectancy,Life Expectancy (AIAN),Life Expectancy (Black),Life Expectancy (Hispanic),Life Expectancy (White),# Deaths,Age-Adjusted Death Rate,Age-Adjusted Mortality (AIAN),Age-Adjusted Mortality (Asian),Age-Adjusted Mortality (Black),Age-Adjusted Mortality (Hispanic),Age-Adjusted Mortality (White),# Child Deaths,Child Mortality Rate,Child Mortality Rate (AIAN),Child Mortality Rate (Asian),Child Mortality Rate (Black),Child Mortality Rate (Hispanic),Child Mortality Rate (White),# Infant Deaths,Infant Mortality Rate,Infant Mortality Rate (AIAN),Infant Mortality Rate (Asian),Infant Mortality Rate (Black),Infant Mortality Rate (Hispanic),Infant Mortality Rate (White),% Frequent Physical Distress,% Frequent Mental Distress,% Adults with Diabetes,# HIV Cases,HIV Prevalence Rate,# Food Insecure,% Food Insecure,# Limited Access to Healthy Foods,% Limited Access to Healthy Foods,# Drug Overdose Deaths,Drug Overdose Mortality Rate,Drug Overdose Mortality Rate (AIAN),Drug Overdose Mortality Rate (Asian),Drug Overdose Mortality Rate (Black),Drug Overdose Mortality Rate (Hispanic),Drug Overdose Mortality Rate (White),% Insufficient Sleep,# Uninsured Adults,% Uninsured Adults,# Uninsured Children,% Uninsured Children,Other Primary Care Provider Rate,Other Primary Care Provider Ratio,Cohort Size,High School Graduation Rate,% Disconnected Youth,Average Grade Performance for 3rd Graders on English Language Arts Standardized Tests,Average Grade Performance for 3rd Graders on English Language Arts Standardized Tests (Asian),Average Grade Performance for 3rd Graders on English Language Arts Standardized Tests (Black),Average Grade Performance for 3rd Graders on English Language Arts Standardized Tests (Hispanic),Average Grade Performance for 3rd Graders on English Language Arts Standardized Tests (White),Average Grade Performance for 3rd Graders on Math Standardized Tests,Average Grade Performance for 3rd Graders on Math Standardized Tests (Asian),Average Grade Performance for 3rd Graders on Math Standardized Tests (Black),Average Grade Performance for 3rd Graders on Math Standardized Tests (Hispanic),Average Grade Performance for 3rd Graders on Math Standardized Tests (White),Segregation Index,Spending per Pupil,School Funding Adequacy,Women's Median Earnings,Men's Median Earnings,Gender Pay Gap,% Enrolled in Free or Reduced Lunch,% Household Income Required for Child Care Expenses,# Child Care Centers,# Suicide Deaths,Suicide Rate (Age-Adjusted),Crude Suicide Rate,Suicide Rate (AIAN),Suicide Rate (Asian),Suicide Rate (Black),Suicide Rate (Hispanic),Suicide Rate (White),# Firearm Fatalities,Firearm Fatalities Rate,Firearm Fatalities Rate (AIAN),Firearm Fatalities Rate (Asian),Firearm Fatalities Rate (Black),Firearm Fatalities Rate (Hispanic),Firearm Fatalities Rate (White),# Motor Vehicle Deaths,Motor Vehicle Mortality Rate,MV Mortality Rate (AIAN),MV Mortality Rate (Asian),MV Mortality Rate (Black),MV Mortality Rate (Hispanic),MV Mortality Rate (White),Non-Petitioned Cases,Petitioned Cases,Denominator,Juvenile Arrest Rate,Traffic Volume,Premature Deaths,Years of Potential Life Lost Rate,YPLL Rate (AIAN),YPLL Rate (Asian),YPLL Rate (Black),YPLL Rate (Hispanic),YPLL Rate (White),% Fair or Poor Health,Average Number of Physically Unhealthy Days,Average Number of Mentally Unhealthy Days,% Low Birthweight,% LBW (AIAN),% LBW (Asian),% LBW (Black),% LBW (Hispanic),% LBW (White),% Adults Reporting Currently Smoking,% Adults with Obesity,Food Environment Index,% Physically Inactive,% With Access to Exercise Opportunities,% Excessive Drinking,# Alcohol-Impaired Driving Deaths,# Driving Deaths,% Driving Deaths with Alcohol Involvement,# Chlamydia Cases,Chlamydia Rate,Teen Birth Rate,Teen Birth Rate (AIAN),Teen Birth Rate (Asian),Teen Birth Rate (Bl

In [14]:
nineteen = pd.read_csv('../../Data Downloads/CountyHealthRankings_2019.csv', dtype = str)
nineteen.set_index(['NAME', 'Year'], inplace = True)

In [15]:
nineteen = nineteen.filter(colstt, axis = 1)

In [16]:
dfs = [data, nineteen]
data = pd.concat(dfs)
data.tail(2)

,,Health Outcomes Ranking,Health Factors Ranking,Length of Life Ranking,Quality of Life Ranking,Health Behaviors Ranking,Clinical Care Ranking,Social & Economic Factors Ranking,Physical Environment Ranking,Life Expectancy,Life Expectancy (AIAN),Life Expectancy (Black),Life Expectancy (Hispanic),Life Expectancy (White),# Deaths,Age-Adjusted Death Rate,Age-Adjusted Mortality (AIAN),Age-Adjusted Mortality (Asian),Age-Adjusted Mortality (Black),Age-Adjusted Mortality (Hispanic),Age-Adjusted Mortality (White),# Child Deaths,Child Mortality Rate,Child Mortality Rate (AIAN),Child Mortality Rate (Asian),Child Mortality Rate (Black),Child Mortality Rate (Hispanic),Child Mortality Rate (White),# Infant Deaths,Infant Mortality Rate,Infant Mortality Rate (AIAN),Infant Mortality Rate (Asian),Infant Mortality Rate (Black),Infant Mortality Rate (Hispanic),Infant Mortality Rate (White),% Frequent Physical Distress,% Frequent Mental Distress,% Adults with Diabetes,# HIV Cases,HIV Prevalence Rate,# Food Insecure,% Food Insecure,# Limited Access to Healthy Foods,% Limited Access to Healthy Foods,# Drug Overdose Deaths,Drug Overdose Mortality Rate,Drug Overdose Mortality Rate (AIAN),Drug Overdose Mortality Rate (Asian),Drug Overdose Mortality Rate (Black),Drug Overdose Mortality Rate (Hispanic),Drug Overdose Mortality Rate (White),% Insufficient Sleep,# Uninsured Adults,% Uninsured Adults,# Uninsured Children,% Uninsured Children,Other Primary Care Provider Rate,Other Primary Care Provider Ratio,Cohort Size,High School Graduation Rate,% Disconnected Youth,Average Grade Performance for 3rd Graders on English Language Arts Standardized Tests,Average Grade Performance for 3rd Graders on English Language Arts Standardized Tests (Asian),Average Grade Performance for 3rd Graders on English Language Arts Standardized Tests (Black),Average Grade Performance for 3rd Graders on English Language Arts Standardized Tests (Hispanic),Average Grade Performance for 3rd Graders on English Language Arts Standardized Tests (White),Average Grade Performance for 3rd Graders on Math Standardized Tests,Average Grade Performance for 3rd Graders on Math Standardized Tests (Asian),Average Grade Performance for 3rd Graders on Math Standardized Tests (Black),Average Grade Performance for 3rd Graders on Math Standardized Tests (Hispanic),Average Grade Performance for 3rd Graders on Math Standardized Tests (White),Segregation Index,Spending per Pupil,School Funding Adequacy,Women's Median Earnings,Men's Median Earnings,Gender Pay Gap,% Enrolled in Free or Reduced Lunch,% Household Income Required for Child Care Expenses,# Child Care Centers,# Suicide Deaths,Suicide Rate (Age-Adjusted),Crude Suicide Rate,Suicide Rate (AIAN),Suicide Rate (Asian),Suicide Rate (Black),Suicide Rate (Hispanic),Suicide Rate (White),# Firearm Fatalities,Firearm Fatalities Rate,Firearm Fatalities Rate (AIAN),Firearm Fatalities Rate (Asian),Firearm Fatalities Rate (Black),Firearm Fatalities Rate (Hispanic),Firearm Fatalities Rate (White),# Motor Vehicle Deaths,Motor Vehicle Mortality Rate,MV Mortality Rate (AIAN),MV Mortality Rate (Asian),MV Mortality Rate (Black),MV Mortality Rate (Hispanic),MV Mortality Rate (White),Non-Petitioned Cases,Petitioned Cases,Denominator,Juvenile Arrest Rate,Traffic Volume,Premature Deaths,Years of Potential Life Lost Rate,YPLL Rate (AIAN),YPLL Rate (Asian),YPLL Rate (Black),YPLL Rate (Hispanic),YPLL Rate (White),% Fair or Poor Health,Average Number of Physically Unhealthy Days,Average Number of Mentally Unhealthy Days,% Low Birthweight,% LBW (AIAN),% LBW (Asian),% LBW (Black),% LBW (Hispanic),% LBW (White),% Adults Reporting Currently Smoking,% Adults with Obesity,Food Environment Index,% Physically Inactive,% With Access to Exercise Opportunities,% Excessive Drinking,# Alcohol-Impaired Driving Deaths,# Driving Deaths,% Driving Deaths with Alcohol Involvement,# Chlamydia Cases,Chlamydia Rate,Teen Birth Rate,Teen Birth Rate (AIAN),Teen Birth Rate (Asian),Teen Birth Rate (Bl

In [17]:
data = data.reset_index(drop = False)

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Columns: 188 entries, NAME to % Census Participation
dtypes: object(188)
memory usage: 705.1+ KB


In [20]:
data.head()

,NAME,Year,Health Outcomes Ranking,Health Factors Ranking,Length of Life Ranking,Quality of Life Ranking,Health Behaviors Ranking,Clinical Care Ranking,Social & Economic Factors Ranking,Physical Environment Ranking,Life Expectancy,Life Expectancy (AIAN),Life Expectancy (Black),Life Expectancy (Hispanic),Life Expectancy (White),# Deaths,Age-Adjusted Death Rate,Age-Adjusted Mortality (AIAN),Age-Adjusted Mortality (Asian),Age-Adjusted Mortality (Black),Age-Adjusted Mortality (Hispanic),Age-Adjusted Mortality (White),# Child Deaths,Child Mortality Rate,Child Mortality Rate (AIAN),Child Mortality Rate (Asian),Child Mortality Rate (Black),Child Mortality Rate (Hispanic),Child Mortality Rate (White),# Infant Deaths,Infant Mortality Rate,Infant Mortality Rate (AIAN),Infant Mortality Rate (Asian),Infant Mortality Rate (Black),Infant Mortality Rate (Hispanic),Infant Mortality Rate (White),% Frequent Physical Distress,% Frequent Mental Distress,% Adults with Diabetes,# HIV Cases,HIV Prevalence Rate,# Food Insecure,% Food Insecure,# Limited Access to Healthy Foods,% Limited Access to Healthy Foods,# Drug Overdose Deaths,Drug Overdose Mortality Rate,Drug Overdose Mortality Rate (AIAN),Drug Overdose Mortality Rate (Asian),Drug Overdose Mortality Rate (Black),Drug Overdose Mortality Rate (Hispanic),Drug Overdose Mortality Rate (White),% Insufficient Sleep,# Uninsured Adults,% Uninsured Adults,# Uninsured Children,% Uninsured Children,Other Primary Care Provider Rate,Other Primary Care Provider Ratio,Cohort Size,High School Graduation Rate,% Disconnected Youth,Average Grade Performance for 3rd Graders on English Language Arts Standardized Tests,Average Grade Performance for 3rd Graders on English Language Arts Standardized Tests (Asian),Average Grade Performance for 3rd Graders on English Language Arts Standardized Tests (Black),Average Grade Performance for 3rd Graders on English Language Arts Standardized Tests (Hispanic),Average Grade Performance for 3rd Graders on English Language Arts Standardized Tests (White),Average Grade Performance for 3rd Graders on Math Standardized Tests,Average Grade Performance for 3rd Graders on Math Standardized Tests (Asian),Average Grade Performance for 3rd Graders on Math Standardized Tests (Black),Average Grade Performance for 3rd Graders on Math Standardized Tests (Hispanic),Average Grade Performance for 3rd Graders on Math Standardized Tests (White),Segregation Index,Spending per Pupil,School Funding Adequacy,Women's Median Earnings,Men's Median Earnings,Gender Pay Gap,% Enrolled in Free or Reduced Lunch,% Household Income Required for Child Care Expenses,# Child Care Centers,# Suicide Deaths,Suicide Rate (Age-Adjusted),Crude Suicide Rate,Suicide Rate (AIAN),Suicide Rate (Asian),Suicide Rate (Black),Suicide Rate (Hispanic),Suicide Rate (White),# Firearm Fatalities,Firearm Fatalities Rate,Firearm Fatalities Rate (AIAN),Firearm Fatalities Rate (Asian),Firearm Fatalities Rate (Black),Firearm Fatalities Rate (Hispanic),Firearm Fatalities Rate (White),# Motor Vehicle Deaths,Motor Vehicle Mortality Rate,MV Mortality Rate (AIAN),MV Mortality Rate (Asian),MV Mortality Rate (Black),MV Mortality Rate (Hispanic),MV Mortality Rate (White),Non-Petitioned Cases,Petitioned Cases,Denominator,Juvenile Arrest Rate,Traffic Volume,Premature Deaths,Years of Potential Life Lost Rate,YPLL Rate (AIAN),YPLL Rate (Asian),YPLL Rate (Black),YPLL Rate (Hispanic),YPLL Rate (White),% Fair or Poor Health,Average Number of Physically Unhealthy Days,Average Number of Mentally Unhealthy Days,% Low Birthweight,% LBW (AIAN),% LBW (Asian),% LBW (Black),% LBW (Hispanic),% LBW (White),% Adults Reporting Currently Smoking,% Adults with Obesity,Food Environment Index,% Physically Inactive,% With Access to Exercise Opportunities,% Excessive Drinking,# Alcohol-Impaired Driving Deaths,# Driving Deaths,% Driving Deaths with Alcohol Involvement,# Chlamydia Cases,Chlamydia Rate,Teen Birth Rate,Teen Birth Rate (AIAN),Teen Birth Rate (Asian),Teen Birth

In [19]:
#export to the SQLite database
conn = sq.connect('../../Outputs/Dem_Transpo_Housing_Collection.db')
data.to_sql('UWisconsin_CountyHealthRankings_20192023', conn, if_exists = 'replace', index = False)

480